# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import seaborn as sns
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV

import pickle

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Roald\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Roald\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Roald\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///Disaster-pipe.db')
df = pd.read_sql_table("messages", engine)
# targets = pd.read_sql_table("categories", engine)

df = df[df.related == 1]


In [4]:
X = df.message.values
Y = df.drop(['message', 'id', 'original', 'genre', 'related'], axis=1).values
target_names = df.drop(['message', 'id', 'original', 'genre', 'related'], axis=1).columns

In [5]:
print(X.shape)
print(Y.shape)

(19906,)
(19906, 35)


### 2. Write a tokenization function to process your text data

In [6]:

def tokenize(text):
    """Takes a text as input an returns a list of tokenized words"""
    stop_words = stopwords.words("english")
    text = re.sub(r"[^a-zA-Z0-9]", " ", text).lower().strip()
    words = word_tokenize(text)
    clean_words = [w for w in words if w not in stopwords.words("english")]
    tokens = [WordNetLemmatizer().lemmatize(w) for w in words if w not in stop_words]
    return [PorterStemmer().stem(w) for w in tokens]

    return clean_tokens
    

In [7]:
clean = tokenize(X[0])
print(clean)

['weather', 'updat', 'cold', 'front', 'cuba', 'could', 'pa', 'haiti']


### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=1)),  
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

pipeline.fit(X_train, Y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
accuracy = (y_pred == Y_test).mean()
print(accuracy)

0.9354370093295348


In [13]:
for i in y_pred:
    print(classification_report(Y_test, y_pred, target_names=target_names))
    break

                        precision    recall  f1-score   support

               request       0.75      0.52      0.61      1476
                 offer       0.00      0.00      0.00        33
           aid_related       0.74      0.74      0.74      3603
          medical_help       0.62      0.15      0.24       701
      medical_products       0.66      0.09      0.16       442
     search_and_rescue       0.38      0.02      0.04       258
              security       0.29      0.01      0.02       157
              military       0.73      0.16      0.26       316
           child_alone       0.00      0.00      0.00         0
                 water       0.85      0.35      0.50       581
                  food       0.84      0.53      0.65       939
               shelter       0.87      0.32      0.47       793
              clothing       0.70      0.14      0.23       139
                 money       0.59      0.07      0.12       188
        missing_people       0.00      

C:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [14]:
pred = pipeline.predict([r"help i need water and food, im dying"])
pd.set_option('display.max_columns', len(target_names))
pred_df = pd.DataFrame(pred, columns=target_names)
pred_df

,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,water,food,shelter,clothing,money,missing_people,refugees,death,other_aid,infrastructure_related,transport,buildings,electricity,tools,hospitals,shops,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
sorted(pipeline.get_params().keys())

In [ ]:
parameters = {'clf__estimator__max_depth': [10, 20, 50],
              'clf__estimator__min_samples_leaf': [1, 2, 4],
              'clf__estimator__min_samples_split': [2, 5, 7],
              'clf__estimator__n_estimators': [100, 500]}
ftwo_scorer = make_scorer(fbeta_score, beta=2)

cv = GridSearchCV(estimator=pipeline, scoring=ftwo_scorer, param_grid=parameters, verbose=10, n_jobs=-1)
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_cv = cv.predict(X_test)
for i in y_pred_cv:
    print(classification_report(Y_test, y_pred_cv, target_names=target_names))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open('disaster_model.pickle', 'wb') as handle:
    pickle.dump(cv, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.